In [3]:
import joblib
import pandas as pd
import unicodedata
import re
import pickle
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, mean_squared_error

In [4]:
pipeline_path = "/Users/fabiancordenod/code/fqbq69/BIMpredict-/notebooks/fabian/randomforestmurspipeline.pkl"
pipeline = joblib.load(pipeline_path)

In [ ]:
# Charger le DataFrame original (remplacez le chemin par le vôtre)
df_test = pd.read_csv("/Users/fabiancordenod/code/fqbq69/BIMpredict-/datatest/test-3/test3/targettest3/murs-3.csv"
, delimiter=';', header=1)  # <-- Remplacez par le vrai chemin du CSV



# Nettoyer les noms de colonnes comme à l'entraînement
def clean_col(col):
    col = ''.join(c for c in unicodedata.normalize('NFD', col) if unicodedata.category(c) != 'Mn')
    col = col.lower()
    col = re.sub(r"[ \-\(\)]", "_", col)
    col = re.sub(r"[^a-z0-9_]", "", col)
    col = re.sub(r"_+", "_", col)
    col = col.strip("_")
    return col

df_test.columns = [clean_col(c) for c in df_test.columns]

In [31]:
df_test

,id,011ec_lot,012ec_ouvrage,013ec_localisation,014ec_mode_constructif,nom,hauteur,epaisseur,ai,as,...,t_teinte,t_finition,t_accessoire,finition,teinte,reference,a_une_association,enrobage_darmature_autres_faces,enrobage_darmature_face_interieure,enrobage_darmature_face_exterieure
0,636502,NaN,NaN,NaN,NaN,EIF_STR - Mur ep. 62,"3,29000000000008","0,62","-2,94000000000002","0,350000000000055",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,636503,NaN,NaN,NaN,NaN,EIF_STR - Mur ep. 62,"3,28867628056769","0,62","-2,94000000000002","0,348676280567673",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,636504,NaN,NaN,NaN,NaN,EIF_STR - Mur ep. 62,"3,11000000000025","0,62","-2,94000000000002","0,170000000000235",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,636505,NaN,NaN,NaN,NaN,EIF_STR - Mur ep. 62,"3,10999999999529","0,62","-2,94000000000002","0,169999999995274",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,636506,NaN,NaN,NaN,NaN,EIF_STR - Mur ep. 62,"3,28999999999948","0,62","-2,94000000000002","0,349999999999459",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980,1430420,NaN,NaN,NaN,NaN,EIF_STR - Mur ep. 16,"1,28","0,16","14,1999999999997","15,4799999999997",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
981,1433089,NaN,NaN,NaN,NaN,EIF_STR - Mur ep. 16,"1,5","0,16","13,9799999999998","15,4799999999998",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
982,1433167,NaN,NaN,NaN,NaN,EIF_STR - Mur ep. 16,"1,48","0,16","13,9799999999998","15,4599999999997",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
983,1433232,NaN,NaN,NaN,NaN,EIF_STR - Mur ep. 16,"1,27931666494601","0,16","14,2006833350537","15,4799999999998",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Les targets à prédire (après nettoyage)
targets = [
    "011ec_lot",
    "012ec_ouvrage",
    "013ec_localisation",
    "014ec_mode_constructif"
]

# Colonnes explicatives attendues par le pipeline
features = pipeline.named_steps['preprocessor'].feature_names_in_

# S'assurer que toutes les colonnes sont présentes
for col in features:
    if col not in df_test.columns:
        df_test[col] = np.nan
        print(f"Colonne manquante ajoutée : {col}")
X_test = df_test[features].copy()


In [19]:
y_pred = pipeline.predict(X_test)

In [22]:
y_pred_df = pd.DataFrame(y_pred, columns=targets)

In [28]:
y_pred_df['013ec_localisation'].value_counts()

INTERIEUR    227
EXTERIEUR     32
Name: 013ec_localisation, dtype: int64

In [ ]:
# Mettre les prédictions dans un DataFrame
df_pred = pd.DataFrame(y_pred, columns=targets)

# Afficher les premières lignes
print(df_pred.head())

# Sauvegarder si besoin
df_pred.to_csv("resultats_predictions_murs1.csv", index=False)